# SVM - Diabetic Retinopathy

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# io related
# from skimage.io import imread
import os
from glob import glob
import cv2
from matplotlib import pyplot as plt
import matplotlib 
from sklearn.model_selection import train_test_split
from PIL import Image

from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.utils import save_image

from skimage import io, transform
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
%matplotlib inline

from sklearn.svm import SVC
import torchvision.transforms as transforms

In [3]:
def load_datasets(base_location = 'F:/UNSW/2022/T2_2022/COMP9417/Project/Data/train.zip.001/train'):

    def severity_level(number):
        if number in [0,1]:
            return 1
        if number == 2:
            return 2
        if number in [3,4]:
            return 3
    
    training_data_name = "train_1"
    train_data_dir = base_location + '/train'
    train_label_file = base_location + '/trainLabels.csv'

    # Load image mapping
    retina_df = pd.read_csv(train_label_file)
    # Get patient ID
    retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
    # Get image path
    retina_df['path'] = retina_df['image'].map(lambda x: train_data_dir + '/' + x + '.jpeg')
    # See if data exists in training data set
    retina_df['exists'] = retina_df['path'].map(os.path.exists)
    print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')

    # Left right eye categorical variable
    # 1 is left eye, 0 is right eye
    retina_df['eye'] = retina_df['image'].map(lambda x: 1 if x.split('_')[-1]=='left' else 0)

    # Remove NA, and keep only the 'existing' images 
    retina_df.dropna(inplace = True)
    retina_df = retina_df[retina_df['exists']]
    
        #EDITTING HERE#
    #retina_df['level'] = retina_df['level'].apply(lambda x: severity_level(x))

    # Split traing and valid sets
    rr_df = retina_df[['PatientId', 'level']].drop_duplicates()
    
    train_ids, valid_ids = train_test_split(rr_df['PatientId'], 
                                    test_size = 0.25, 
                                    #random_state = 2018,
                                    stratify = rr_df['level'])
                                    
    raw_train_df = retina_df[retina_df['PatientId'].isin(train_ids)]
    valid_df = retina_df[retina_df['PatientId'].isin(valid_ids)]
    print('Pre-balance: train', raw_train_df.shape[0], 'validation', valid_df.shape[0])
    
    
    # balance size variance in each class
    #train_df = raw_train_df.groupby(['level', 'eye']).apply(lambda x: x.sample(75, replace = True)).reset_index(drop = True)                                                   
    #print('Post-balance: train', train_df.shape[0], 'validation', valid_df.shape[0])

    return raw_train_df, valid_df

In [4]:
class retinaDataset(Dataset):

    def __init__(self, transforms=None, image_size = 192):
        'Initialization'
        self.image_size = image_size
        self.transforms = transforms        
        self.train_df, self.valid_df = load_datasets()
        
        self.train_df.reset_index(drop = True, inplace = True)
        self.valid_df.reset_index(drop = True, inplace = True)
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.train_df)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        
        img_path = self.train_df["path"][index]
        
        # print(img_path)

        img = Image.open(img_path)
        
        if self.transforms:
            img = self.transforms(img)

        return img, torch.tensor(self.train_df.iloc[index].level)
    
    def testLen(self):
        return len(self.valid_df)
    def getTest(self, index):
        #gets test set 
        img_path = self.valid_df["path"][index]
        
        img = Image.open(img_path)
        
        if(self.transforms):
            img = self.transforms(img)

        return img, torch.tensor(self.valid_df.iloc[index].level)

In [5]:
def generate_results(model, xTrain, yTrain, xTest, yTest, train_flag = True):
    num_dec_point = 3
    #train_flag = True

    # generate predictions
    y_pred = model.predict(xTest)

    if train_flag:
        y_train_pred = model.predict(xTrain) 
        print('Model Training accuracy is: ', accuracy_score(yTrain, y_train_pred))

    # calculate testing accuracy
    accuracy = accuracy_score(yTest, y_pred)
    print('Model Testing accuracy is: ', accuracy)

    p_mic, r_mic, f1_mic, _ = precision_recall_fscore_support(yTest, 
                            y_pred,
                            average='micro',
                            warn_for=())
    p_mac, r_mac, f1_mac, _ = precision_recall_fscore_support(yTest, 
                        y_pred,
                        average='macro',
                        warn_for=())
    print('micro acc,prec,rec,f1: ',round(accuracy,num_dec_point), round(p_mic,num_dec_point), round(r_mic,num_dec_point), round(f1_mic,num_dec_point),sep="\t")
    print('macro prec,rec,f1: ',round(p_mac,num_dec_point), round(r_mac,num_dec_point), round(f1_mac,num_dec_point),sep="\t")
    print('Confusion Matrix is: ', confusion_matrix(yTest, y_pred))

In [7]:
from sklearn.preprocessing import MinMaxScaler

#works better if data is not gaussian distributed
def normalisation(X_train, X_test):
    norm = MinMaxScaler().fit(X_train)
    X_train_norm = norm.transform(X_train)
    X_test_norm = norm.transform(X_test)
    return X_train, X_test

In [8]:
image_size = 192

my_transforms = transforms.Compose([
    transforms.Resize((image_size,image_size)),
    transforms.ToTensor(),
])

dataset = retinaDataset(transforms=my_transforms, image_size = image_size)

8408 images found of 35126 total
Pre-balance: train 6518 validation 2318


In [9]:
dataset_flag = True  #'''getting all of the train_df into xTrain and valid_df into xTest'''

#shuffle dataset
import random

if dataset_flag == False:
    n = 6000 #number of total datasets
    ratio = 0.8 #training:testing
    #total = random.sample(range(len(dataset)), n)
    training_number = int(n * ratio)
    testing_number = n - training_number
    
    training_items = random.sample(range(len(dataset)), training_number)
    testing_items = random.sample(range(dataset.testLen()), testing_number)
    print('training size: ', training_number, 'testing size: ', testing_number)
else:
    from PIL import ImageFile
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    training_items = random.sample(range(len(dataset)), len(dataset))
    testing_items = random.sample(range(dataset.testLen()), dataset.testLen())
    print('training size: ', len(training_items), 'testing size: ', len(testing_items))

xTrain = []
yTrain = []
xTest = []
yTest = []

# Populate xTrain yTrain
for index, i in enumerate(training_items):
    if index % 10 == 0:
        print(str(index) + ': train /' + str(len(training_items)))
    item, label = dataset.__getitem__(i)
    oneD = item.flatten()
    xTrain.append(oneD)
    # Append labels
    yTrain.append(label)
    
# Populate xTest yTest
for index, i in enumerate(testing_items):
    if index % 10 == 0:
        print(str(index) + ': test /' + str(len(testing_items)))
    item, label = dataset.getTest(i)
    oneD = item.flatten()
    # Append flatten image matrix
    xTest.append(oneD)
    # Append labels
    yTest.append(label) 
    
xTrain = np.vstack(xTrain)
yTrain = np.array(yTrain)
xTest = np.vstack(xTest)
yTest = np.array(yTest)

print('normalising')
xTrain, xTest = normalisation(xTrain, xTest) #normalisation

print('Completed.')

training size:  6518 testing size:  2318
0: train /6518
10: train /6518
20: train /6518
30: train /6518
40: train /6518
50: train /6518
60: train /6518
70: train /6518
80: train /6518
90: train /6518
100: train /6518
110: train /6518
120: train /6518
130: train /6518
140: train /6518
150: train /6518
160: train /6518
170: train /6518
180: train /6518
190: train /6518
200: train /6518
210: train /6518
220: train /6518
230: train /6518
240: train /6518
250: train /6518
260: train /6518
270: train /6518
280: train /6518
290: train /6518
300: train /6518
310: train /6518
320: train /6518
330: train /6518
340: train /6518
350: train /6518
360: train /6518
370: train /6518
380: train /6518
390: train /6518
400: train /6518
410: train /6518
420: train /6518
430: train /6518
440: train /6518
450: train /6518
460: train /6518
470: train /6518
480: train /6518
490: train /6518
500: train /6518
510: train /6518
520: train /6518
530: train /6518
540: train /6518
550: train /6518
560: train /6518
5

4600: train /6518
4610: train /6518
4620: train /6518
4630: train /6518
4640: train /6518
4650: train /6518
4660: train /6518
4670: train /6518
4680: train /6518
4690: train /6518
4700: train /6518
4710: train /6518
4720: train /6518
4730: train /6518
4740: train /6518
4750: train /6518
4760: train /6518
4770: train /6518
4780: train /6518
4790: train /6518
4800: train /6518
4810: train /6518
4820: train /6518
4830: train /6518
4840: train /6518
4850: train /6518
4860: train /6518
4870: train /6518
4880: train /6518
4890: train /6518
4900: train /6518
4910: train /6518
4920: train /6518
4930: train /6518
4940: train /6518
4950: train /6518
4960: train /6518
4970: train /6518
4980: train /6518
4990: train /6518
5000: train /6518
5010: train /6518
5020: train /6518
5030: train /6518
5040: train /6518
5050: train /6518
5060: train /6518
5070: train /6518
5080: train /6518
5090: train /6518
5100: train /6518
5110: train /6518
5120: train /6518
5130: train /6518
5140: train /6518
5150: trai

## Fitting and Predicting SVM model

### 26/7/2022 - Best model

with 5 classes, and 6486, 2310 different gamma - with normalization, no class balance

In [10]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9367904265111997
Model Testing accuracy is:  0.7359792924935289
micro acc,prec,rec,f1: 	0.736	0.736	0.736	0.736
macro prec,rec,f1: 	0.659	0.479	0.538
Confusion Matrix is:  [[1446   42  109    6   12]
 [ 106   90    5    0    0]
 [ 245    8  127    0    2]
 [  36    2    4   19    1]
 [  31    1    2    0   24]]


In [12]:
import pickle

# save
with open('model.pkl','wb') as f:
    pickle.dump(svm, f, protocol=4)



### 25/7/2022

with 5 classes, and 2800, 700 different gamma - with normalization, no class balance

In [40]:
svm = SVC(kernel='rbf', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7207142857142858
Model Testing accuracy is:  0.6857142857142857
micro acc,prec,rec,f1: 	0.686	0.686	0.686	0.686
macro prec,rec,f1: 	0.137	0.2	0.163
Confusion Matrix is:  [[480   0   0   0   0]
 [ 67   0   0   0   0]
 [128   0   0   0   0]
 [ 15   0   0   0   0]
 [ 10   0   0   0   0]]


In [41]:
svm = SVC(kernel='poly', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7207142857142858
Model Testing accuracy is:  0.6857142857142857
micro acc,prec,rec,f1: 	0.686	0.686	0.686	0.686
macro prec,rec,f1: 	0.137	0.2	0.163
Confusion Matrix is:  [[480   0   0   0   0]
 [ 67   0   0   0   0]
 [128   0   0   0   0]
 [ 15   0   0   0   0]
 [ 10   0   0   0   0]]


In [42]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9757142857142858
Model Testing accuracy is:  0.6742857142857143
micro acc,prec,rec,f1: 	0.674	0.674	0.674	0.674
macro prec,rec,f1: 	0.477	0.37	0.402
Confusion Matrix is:  [[424   9  38   5   4]
 [ 46  16   5   0   0]
 [ 97   5  25   1   0]
 [ 10   0   0   5   0]
 [  6   2   0   0   2]]


with 5 classes, and 720, 180, different gamma - with normalization

In [17]:
svm = SVC(kernel='linear', C=0.1, gamma=1.0, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9944444444444445
Model Testing accuracy is:  0.5277777777777778
micro acc,prec,rec,f1: 	0.528	0.528	0.528	0.528
macro prec,rec,f1: 	0.206	0.202	0.197
Confusion Matrix is:  [[89  9 14  0  1]
 [17  4  3  1  0]
 [29  0  2  1  0]
 [ 4  0  1  0  1]
 [ 2  0  2  0  0]]


In [18]:
svm = SVC(kernel='linear', C=0.1, gamma=0.1, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9944444444444445
Model Testing accuracy is:  0.5277777777777778
micro acc,prec,rec,f1: 	0.528	0.528	0.528	0.528
macro prec,rec,f1: 	0.206	0.202	0.197
Confusion Matrix is:  [[89  9 14  0  1]
 [17  4  3  1  0]
 [29  0  2  1  0]
 [ 4  0  1  0  1]
 [ 2  0  2  0  0]]


In [19]:
svm = SVC(kernel='linear', C=0.1, gamma=0.01, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9944444444444445
Model Testing accuracy is:  0.5277777777777778
micro acc,prec,rec,f1: 	0.528	0.528	0.528	0.528
macro prec,rec,f1: 	0.206	0.202	0.197
Confusion Matrix is:  [[89  9 14  0  1]
 [17  4  3  1  0]
 [29  0  2  1  0]
 [ 4  0  1  0  1]
 [ 2  0  2  0  0]]


In [20]:
svm = SVC(kernel='linear', C=0.1, gamma=0.001, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9944444444444445
Model Testing accuracy is:  0.5277777777777778
micro acc,prec,rec,f1: 	0.528	0.528	0.528	0.528
macro prec,rec,f1: 	0.206	0.202	0.197
Confusion Matrix is:  [[89  9 14  0  1]
 [17  4  3  1  0]
 [29  0  2  1  0]
 [ 4  0  1  0  1]
 [ 2  0  2  0  0]]


In [21]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9944444444444445
Model Testing accuracy is:  0.5277777777777778
micro acc,prec,rec,f1: 	0.528	0.528	0.528	0.528
macro prec,rec,f1: 	0.206	0.202	0.197
Confusion Matrix is:  [[89  9 14  0  1]
 [17  4  3  1  0]
 [29  0  2  1  0]
 [ 4  0  1  0  1]
 [ 2  0  2  0  0]]


In [22]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9958333333333333
Model Testing accuracy is:  0.5611111111111111
micro acc,prec,rec,f1: 	0.561	0.561	0.561	0.561
macro prec,rec,f1: 	0.215	0.208	0.2
Confusion Matrix is:  [[96  6 10  0  1]
 [17  4  2  2  0]
 [29  1  1  1  0]
 [ 4  0  1  0  1]
 [ 3  0  1  0  0]]


In [24]:
svm._gamma

9.04224537037037e-06

poly

In [27]:
svm = SVC(kernel='poly', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


rbf

In [25]:
svm = SVC(kernel='rbf', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [26]:
svm = SVC(kernel='rbf', C=0.1, gamma='auto', class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.09027777777777778
Model Testing accuracy is:  0.1388888888888889
micro acc,prec,rec,f1: 	0.139	0.139	0.139	0.139
macro prec,rec,f1: 	0.028	0.2	0.049
Confusion Matrix is:  [[  0 113   0   0   0]
 [  0  25   0   0   0]
 [  0  32   0   0   0]
 [  0   6   0   0   0]
 [  0   4   0   0   0]]


conclusion: class_weight = balanced is bad

In [28]:
svm = SVC(kernel='rbf', C=1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [29]:
svm = SVC(kernel='rbf', C=0.01, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [30]:
svm = SVC(kernel='rbf', C=0.001, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [31]:
svm = SVC(kernel='rbf', C=0.0001, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.0001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [32]:
svm = SVC(kernel='rbf', C=0.00001, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=1e-05, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [33]:
svm = SVC(kernel='rbf', C=0.000001, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=1e-06, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [34]:
svm = SVC(kernel='rbf', C=2, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [35]:
svm = SVC(kernel='rbf', C=5, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


In [36]:
svm = SVC(kernel='rbf', C=10, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.7263888888888889
Model Testing accuracy is:  0.6277777777777778
micro acc,prec,rec,f1: 	0.628	0.628	0.628	0.628
macro prec,rec,f1: 	0.126	0.2	0.154
Confusion Matrix is:  [[113   0   0   0   0]
 [ 25   0   0   0   0]
 [ 32   0   0   0   0]
 [  6   0   0   0   0]
 [  4   0   0   0   0]]


with 5 classes, and 320, 80, different gamma - with normalization

In [13]:
svm = SVC(kernel='linear', C=0.1, gamma=1.0, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)
generate_results(svm, xTrain_norm, yTrain, xTest_norm, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.6125
micro acc,prec,rec,f1: 	0.612	0.612	0.612	0.612
macro prec,rec,f1: 	0.306	0.253	0.265
Confusion Matrix is:  [[44  2  5  2  0]
 [ 6  2  1  0  0]
 [ 9  0  3  1  1]
 [ 1  0  0  0  0]
 [ 3  0  0  0  0]]


with 5 classes, and 320, 80, different gamma

In [7]:
svm = SVC(kernel='linear', C=0.1, gamma=1.0, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.186	0.187	0.186
Confusion Matrix is:  [[38  4 11  2  2]
 [ 2  0  1  0  0]
 [11  0  4  0  0]
 [ 3  0  0  0  0]
 [ 2  0  0  0  0]]


In [8]:
svm = SVC(kernel='linear', C=0.1, gamma=0.5, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.186	0.187	0.186
Confusion Matrix is:  [[38  4 11  2  2]
 [ 2  0  1  0  0]
 [11  0  4  0  0]
 [ 3  0  0  0  0]
 [ 2  0  0  0  0]]


In [9]:
svm = SVC(kernel='linear', C=0.1, gamma=0.1, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.186	0.187	0.186
Confusion Matrix is:  [[38  4 11  2  2]
 [ 2  0  1  0  0]
 [11  0  4  0  0]
 [ 3  0  0  0  0]
 [ 2  0  0  0  0]]


In [10]:
svm = SVC(kernel='linear', C=0.1, gamma=0.01, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.186	0.187	0.186
Confusion Matrix is:  [[38  4 11  2  2]
 [ 2  0  1  0  0]
 [11  0  4  0  0]
 [ 3  0  0  0  0]
 [ 2  0  0  0  0]]


In [11]:
svm = SVC(kernel='linear', C=0.1, gamma=0.001, class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.186	0.187	0.186
Confusion Matrix is:  [[38  4 11  2  2]
 [ 2  0  1  0  0]
 [11  0  4  0  0]
 [ 3  0  0  0  0]
 [ 2  0  0  0  0]]


## History

with 5 classes, and 320, 80

In [13]:
svm = SVC(kernel='linear', C=0.1, gamma='scale', class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.99375
Model Testing accuracy is:  0.5875
micro acc,prec,rec,f1: 	0.588	0.588	0.588	0.588
macro prec,rec,f1: 	0.164	0.181	0.171
Confusion Matrix is:  [[46  3  6  1  0]
 [ 6  0  1  0  0]
 [11  0  1  0  0]
 [ 2  0  0  0  0]
 [ 1  2  0  0  0]]


with 3 classes, and 320, 80

In [8]:
svm = SVC(kernel='linear', C=0.1, gamma='scale', class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  1.0
Model Testing accuracy is:  0.525
micro acc,prec,rec,f1: 	0.525	0.525	0.525	0.525
macro prec,rec,f1: 	0.296	0.303	0.289
Confusion Matrix is:  [[38 10  2]
 [22  4  1]
 [ 3  0  0]]


720, 180 = 900

In [10]:
svm = SVC(kernel='linear', C=0.1, gamma='scale', class_weight = 'balanced', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(svm, xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9875
Model Testing accuracy is:  0.6222222222222222
micro acc,prec,rec,f1: 	0.622	0.622	0.622	0.622
macro prec,rec,f1: 	0.345	0.291	0.304
Confusion Matrix is:  [[100   8  10   0   1]
 [ 18   1   1   0   0]
 [ 21   1  10   0   0]
 [  4   0   1   0   0]
 [  2   0   1   0   1]]


1440, 360 = 1800

In [22]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9840277777777777
Model Testing accuracy is:  0.6138888888888889
micro acc,prec,rec,f1: 	0.614	0.614	0.614	0.614
macro prec,rec,f1: 	0.278	0.235	0.237
Confusion Matrix is:  [[206   8  25   1   2]
 [ 32   2   1   1   0]
 [ 46   0  12   1   0]
 [ 11   1   2   1   0]
 [  5   1   2   0   0]]


2160, 540

In [15]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9819444444444444
Model Testing accuracy is:  0.6592592592592592
micro acc,prec,rec,f1: 	0.659	0.659	0.659	0.659
macro prec,rec,f1: 	0.543	0.301	0.335
Confusion Matrix is:  [[326  15  36   1   0]
 [ 35   8   1   1   0]
 [ 64   2  18   0   0]
 [ 12   3   2   2   0]
 [  7   0   4   1   2]]


720, 180

In [13]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9986111111111111
Model Testing accuracy is:  0.6888888888888889
micro acc,prec,rec,f1: 	0.689	0.689	0.689	0.689
macro prec,rec,f1: 	0.237	0.239	0.237
Confusion Matrix is:  [[117   5  14   0   1]
 [  7   1   2   0   0]
 [ 19   0   6   0   0]
 [  3   1   1   0   0]
 [  2   0   1   0   0]]


train 480, 120

In [11]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9979166666666667
Model Testing accuracy is:  0.625
micro acc,prec,rec,f1: 	0.625	0.625	0.625	0.625
macro prec,rec,f1: 	0.19	0.194	0.192
Confusion Matrix is:  [[72  7 11  1  1]
 [ 8  0  2  0  0]
 [12  1  3  0  0]
 [ 1  0  1  0  0]
 [ 0  0  0  0  0]]


training = 720, testing 180

In [66]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.9986111111111111
Model Testing accuracy is:  0.3611111111111111
micro acc,prec,rec,f1: 	0.361	0.361	0.361	0.361
macro prec,rec,f1: 	0.271	0.269	0.231
Confusion Matrix is:  [[44 25 24 17 13]
 [ 4 12  6  1  3]
 [ 2  7  8  3  1]
 [ 0  4  1  1  1]
 [ 1  1  1  0  0]]


In [67]:
svm = SVC(kernel='poly', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.2013888888888889
Model Testing accuracy is:  0.11666666666666667
micro acc,prec,rec,f1: 	0.117	0.117	0.117	0.117
macro prec,rec,f1: 	0.023	0.2	0.042
Confusion Matrix is:  [[  0   0 123   0   0]
 [  0   0  26   0   0]
 [  0   0  21   0   0]
 [  0   0   7   0   0]
 [  0   0   3   0   0]]


In [57]:
svm = SVC(kernel='linear', C=1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  1.0
Model Testing accuracy is:  0.2875
micro acc,prec,rec,f1: 	0.288	0.288	0.288	0.288
macro prec,rec,f1: 	0.204	0.329	0.171
Confusion Matrix is:  [[19 17  7  9  9]
 [ 5  2  0  2  0]
 [ 3  1  1  3  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  1]]


In [58]:
svm = SVC(kernel='linear', C=0.5, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  1.0
Model Testing accuracy is:  0.2875
micro acc,prec,rec,f1: 	0.288	0.288	0.288	0.288
macro prec,rec,f1: 	0.204	0.329	0.171
Confusion Matrix is:  [[19 17  7  9  9]
 [ 5  2  0  2  0]
 [ 3  1  1  3  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  1]]


In [59]:
svm = SVC(kernel='linear', C=0.1, gamma='auto', probability=False) #faster when probability = False

# fit model
svm.fit(xTrain, yTrain)
print(svm)

generate_results(xTrain, yTrain, xTest, yTest, train_flag = True)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Model Training accuracy is:  0.996875
Model Testing accuracy is:  0.2875
micro acc,prec,rec,f1: 	0.288	0.288	0.288	0.288
macro prec,rec,f1: 	0.207	0.329	0.172
Confusion Matrix is:  [[19 17  6 10  9]
 [ 5  2  0  2  0]
 [ 3  1  1  3  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  1]]
